In [ ]:
import typing as tp
from matplotlib.image import imread
from matplotlib import pyplot as plt
import torch
import torchvision.transforms.functional as fn
from torchvision import transforms
from sklearn.model_selection import train_test_split
import os
import sys
import sklearn
from tqdm import tqdm
from torch import nn, Tensor
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import random

torch.manual_seed(42)
random.seed(42)

In [ ]:
dataset_dir = "/local/tmp/BostonGene_application_data/images/"
processed_image_size = 64
device = "cuda"

In [ ]:
# targets = os.listdir(dataset_dir)
# _filenames = {target: [file for file in os.listdir(f"{dataset_dir}/{target}")] for target in targets}

Let's start by writing a wrapper class for our dataset

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir: str, image_size: int = 64, train: bool = True, manual_ids: tp.Optional[bool] = None,
                 augmentations=None):
        """
        A wrapper class for our dataset.

        Since our dataset is small, we simply store it in RAM.

        :param dataset_dir: location of the upacked dataset
        :param image_size: size, to which the image will be resized during preprocessing of the image
        :param train: set True for train, or False for test
        :param manual_ids: override default train\test ids split by manually choosing which ids to load. Ids should be chosen from MyDataset.get_ids()
        """
        super().__init__()
        self._train = train
        self.target_names = os.listdir(dataset_dir)
        self.target_encoding = {self.target_names[_id]: _id for _id in range(len(self.target_names))}

        targets_and_filenames = [(target, filename) for target in self.target_names for filename in
                                 os.listdir(f"{dataset_dir}/{target}")]

        if manual_ids is None:
            targets = [self.target_encoding[item[0]] for item in targets_and_filenames]
            train_ids, test_ids = train_test_split(list(range(len(targets_and_filenames))), random_state=42,
                                                   test_size=0.1, shuffle=True, stratify=targets)
            ids = train_ids if self._train else test_ids
        else:
            ids = manual_ids

        # Implementing augmentations
        if self._train and augmentations is not None:
            self.is_augmented = True
            preprocessing_pipeline = transforms.Compose([
                transforms.Lambda(lambda img: img / 255),  # Normalize
            ])
            self.augmentation_pipeline = transforms.Compose([
                augmentations,
                transforms.Lambda(lambda img: transforms.RandomCrop(size=min(img.shape[1:]))(img)),
                # Randomly crop to square
                transforms.Resize(size=image_size)
            ])
        else:
            self.is_augmented = False
            preprocessing_pipeline = transforms.Compose([
                transforms.Lambda(lambda img: img / 255),  # Normalize
                transforms.Lambda(lambda img: transforms.CenterCrop(size=min(img.shape[1:]))(img)),  # Crop to square
                transforms.Resize(size=image_size)
            ])

        self._data = []
        self._targets = []
        for target, file in np.array(targets_and_filenames)[ids]:
            self._data.append(preprocessing_pipeline(
                torch.tensor(imread(f"{dataset_dir}/{target}/{file}"), dtype=torch.float32).permute(2, 1, 0)))
            self._targets.append(self.target_encoding[target])

    def __len__(self):
        return len(self._targets)

    def __getitem__(self, item):
        img = self.augmentation_pipeline(self._data[item]) if self.is_augmented else self._data[item]
        return img, self._targets[item]

    @staticmethod
    def get_ids(dataset_dir: str):
        return list(range(sum([len(os.listdir(f"{dataset_dir}/{target}")) for target in os.listdir(f"{dataset_dir}")])))

In [ ]:
train_dataset = MyDataset(dataset_dir, processed_image_size, train=True,
                          augmentations=transforms.Compose([
                              transforms.RandomRotation(10),
                              transforms.RandomHorizontalFlip(p=0.1),
                              transforms.RandomVerticalFlip(p=0.1)
                          ])
                          )
test_dataset = MyDataset(dataset_dir, processed_image_size, train=False)

In [ ]:
img_id = 1
print("Before:")
plt.imshow(train_dataset._data[img_id].permute(2, 1, 0))
plt.show()
print("After:")
plt.imshow(train_dataset[img_id][0].permute(2, 1, 0))
plt.show()

In [ ]:
# Let's create a simple baseline - a ResNet model
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.activation = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

        self.downsample = False
        if in_channels != out_channels:
            self.downsample = True
            self.downsample_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                              kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, _input: Tensor) -> Tensor:
        identity = _input
        if self.downsample:
            identity = self.downsample_layer(identity)

        output = self.conv1(_input)
        output = self.bn1(output)
        output = self.activation(output)
        output = self.conv2(output)
        output = self.bn2(output)
        output += identity
        output = self.activation(output)

        return output


class MyResNet(nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, input_channels: int, layers: list[tuple[int, int]], num_of_classes: int, initial_layers:
    tp.Optional[nn.Module] = None):
        super().__init__()

        if initial_layers is None:
            initial_layers = nn.Identity()
        self.initial_layers = initial_layers

        self.layers = nn.Sequential()
        for layer_id, layer in enumerate(layers):
            num_of_blocks, channels_per_block = layer
            for block_id in range(num_of_blocks):
                if block_id == 0:
                    if layer_id == 0:
                        in_channels = input_channels
                    else:
                        in_channels = layers[layer_id - 1][1]
                else:
                    in_channels = channels_per_block
                self.layers.add_module(f"ResBlock{layer_id}_{block_id}",
                                       ResBlock(in_channels=in_channels, out_channels=channels_per_block))

            if layer_id != len(layers) - 1:
                self.layers.add_module(f"pool{layer_id}", nn.MaxPool2d(2))

        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=layers[-1][1], out_features=num_of_classes)

    def forward(self, _input: Tensor) -> Tensor:
        output = self.initial_layers(_input)
        output = self.layers(output)
        output = self.avg_pool(output)
        output = torch.flatten(output, 1)
        output = self.fc(output)

        return output

In [ ]:
# Напишем стандартный код, который можно использовать для обучения моделек:

def trainer(number_of_epochs,
            dataset,
            val_dataset,
            batch_size,
            model,
            loss_function,
            optimizer,
            writer,
            lr=0.001,
            lr_multiplier_schedule=None):
    def make_val_report(iteration):
        report = calculate_val_performance(model, DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=8))

        def report_avg(which_avg):
            for metric in report[which_avg]:
                if metric != 'support':
                    writer.add_scalar(f'metrics/{which_avg}/{metric}', report[which_avg][metric], iteration)

        report_avg('weighted avg')
        report_avg('macro avg')
        writer.add_scalar(f'metrics/accuracy', report['accuracy'], iteration)

    optima = optimizer(model.parameters(), lr=lr)

    iterations = tqdm(range(number_of_epochs), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:
        if lr_multiplier_schedule is not None and it in lr_multiplier_schedule:
            lr *= lr_multiplier_schedule[it]
            optima = optimizer(model.parameters(), lr=lr)

        epoch_loss = train_epoch(train_generator=DataLoader(dataset, batch_size, shuffle=True, num_workers=8),
                                 model=model,
                                 loss_function=loss_function,
                                 optimizer=optima)

        iterations.set_postfix({'train epoch loss': epoch_loss})
        writer.add_scalar('metrics/train_loss', epoch_loss, it)
        make_val_report(it)


def train_epoch(train_generator, model, loss_function, optimizer):
    model.train()

    epoch_loss = 0
    total = 0
    for x, y in train_generator:
        optimizer.zero_grad()

        output = model(x.to(model.device))

        loss = loss_function(output, y.to(model.device))
        loss.backward()

        optimizer.step()

        epoch_loss += loss.cpu().item()
        total += 1

    return epoch_loss / total


def calculate_val_performance(model, val_dataset):
    model.eval()

    y_pred = [int(torch.argmax(model(x.to(model.device)).cpu())) for x, y in val_dataset]
    y_target = [int(y) for x, y in val_dataset]

    return sklearn.metrics.classification_report(y_target, y_pred, output_dict=True, zero_division=0)


In [ ]:
model = MyResNet(input_channels=3, layers=[(4, 64), (4, 128)], num_of_classes=8)

In [ ]:
model.to('cuda')
trainer(number_of_epochs=10,
        dataset=train_dataset,
        val_dataset=test_dataset,
        batch_size=16,
        model=model,
        loss_function=nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam,
        writer=SummaryWriter(log_dir='/local/tmp/logs/BG_application/resnet'),
        lr=0.001,
        lr_multiplier_schedule={1: 0.5, 3: 0.5, 5: 0.5, 7: 0.5})

In [ ]:
# 0.42-0.44